### Imports

In [5]:
import cv2
from time import sleep, time
import json
import numpy as np
import base64
import grpc
import logging
import zlib
import struct
import math
from datetime import datetime, timedelta
import pravega
from ipywebrtc import CameraStream, ImageRecorder

## Open camera

In [6]:
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
camera

CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, 'height': 480}})

### Helper functions

In [7]:
def chunks(l, n):
    """Yield successive n-sized chunks from l.
    From https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks"""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def pravega_chunker_v1(payload, max_chunk_size):
    """Split payload into chunks of 1 MiB or less.
    When written to Pravega, chunked events have the following 64-bit header.
      version: value must be 1 (8-bit signed integer)
      reserved: value must be 0 (3 8-bit signed integers)
      chunk_index: 0-based chunk index (16-bit signed big endian integer)
      final_chunk_index: number of chunks minus 1 (16 bit signed big endian integer)
    """
    version = 1
    max_chunk_data_size = max_chunk_size - 4096
    chunk_list = list(chunks(payload, max_chunk_data_size))
    final_chunk_index = len(chunk_list) - 1
    for chunk_index, chunked_payload in enumerate(chunk_list):
        is_final_chunk = chunk_index == final_chunk_index
        header = struct.pack('!bxxxhh', version, chunk_index, final_chunk_index)
        chunked_event = header + chunked_payload
        yield (chunked_event, chunk_index, is_final_chunk)

def encode_record(record: dict) -> bytes:
    """Encode the record
    JSON is universally compatible but require images to be base64 encoded.
    For optimal performance, other encodings should be used such as Avro or Protobuf.
    """
    r = record.copy()
    r['data'] = base64.b64encode(record['data']).decode('utf-8')
    encoded = json.dumps(r).encode('utf-8')
    return encoded

### Generate a record for every frame

In [8]:
def pravega_request_generator(data_generator, scope, stream, max_chunk_size, use_transactions):
    for record in data_generator:
        t = record['timestamp']
        sleep_sec = t/1000.0 - time()
        if sleep_sec > 0.0:
            sleep(sleep_sec)
        record['timestamp'] = (datetime(1970, 1, 1) + timedelta(milliseconds=t)).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + 'Z'
        payload = encode_record(record)
        for (chunked_event, chunk_index, is_final_chunk) in pravega_chunker_v1(payload, max_chunk_size=max_chunk_size):
            request = pravega.pb.WriteEventsRequest(
                event=chunked_event,
                scope=scope,
                stream=stream,
                use_transaction=use_transactions,
                commit=is_final_chunk and use_transactions,
                )
            # logging.info(request)
            yield request
        if True:
            record_to_log = record.copy()
            # record_to_log['data'] = str(record_to_log['data'][:10]) + '...'
            del record_to_log['data']
            record_to_log['data_len'] = len(record['data'])
            record_to_log['payload_len'] = len(payload)
            record_to_log['final_chunk_index'] = chunk_index
            logging.info('%d: %s' % (record_to_log['camera'], json.dumps(record_to_log)))

### Read one frame at a time from camera feed and create a record

In [34]:
def data_generator(image_recorder, include_checksum, ssrc):
    frame_number = 0
    while True:
        sleep(2)
        # Capture video frame by frame
        # ret, frame = cap.read()
        timestamp = int(time() * 1000)
        # _, data = cv2.imencode(".png", frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        #_, data = cv2.imencode(".jpg", frame)
        image_recorder.recording = True
        image_recorder.autosave = False
        data = image_recorder.image.value
        if include_checksum:
            # Add CRC32 checksum to allow for error detection.
            chucksum = struct.pack('!I', zlib.crc32(data))
            data = chucksum + data
        record = {
            'timestamp': timestamp,
            'frame_number': frame_number,
            'camera': 0,
            'ssrc': ssrc,
            'data': data            
        }
        yield record
        frame_number += 1

In [35]:
image_recorder = ImageRecorder(stream=camera)

In [40]:
next(data_generator(image_recorder, False, 1))

{'timestamp': 1558752405910,
 'frame_number': 0,
 'camera': 0,
 'ssrc': 1,
 'data': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc\xe4\x00\x00 \x00IDATx\x9c\xec\xbd\xe7WS\xff\xda\xee\xbb\xfe\xd93\xce\x8bS\x9e\xf5SQJH\xef\xbd\xf7NB\x12z\x13, \n\xa8\xa0\x80\x8a\xd8P\x8a\xf4\x12B\r]?\xfb\xc57\x99\x80k=\xfb\xd9\xcf\xde\xfb\xecq\xce8k\x8eq\x8d\xf9\xcdd\x06\x95L\xc9g^\xd7}\xdf\xf9[Po$l0\xe1Q[\xf1il\xb8\x8d!|\xd6(Ns\x1c\xb75\x89\xc7\x9e\xc6\xebh\xc2k\t\xe11\x07\xf1[|\x84\xec!B\xb6\x18qw\x8a\x98?C\xc4\x9b\xc2\xe7K\xe2\xf1&\xf0ybx\xddQB\x9e8^[\x10\x9f5\x8c\xcf\x1a&\xe4H\x12r$\t8+rD\x88z\x12D\xdcq\xa2\x9e\x041_\x9c\xa87F\xc8\x13%\x16H\x12\t&\x89\x86R\x04\xbdQ\x02\x9e\x08\xe1@\x9ch(E"\xd2D2\x9a!\x15\xcb\x12\x0f\xa7\x89G\xe2$\xa2\t\x92\xb1$\x89h\x82hH(\x1eN\x93\x8cfhJ\xb4\xd1\x9c\xea\xa09\xdbAS\xaa\x95LS\x9el\xb6@!\xdfNss\x0b\xd9l\x81l\xb6@s\xbe\x8d\\\xa1\xfd\xd6\xf1\x96B\x07-\xf9V\xda\nm\xb4\xb4\xb4\xd1\xd6\xd6AK\xa1\x83B^\x9c\xd7R\xe8\xa0\xb3\x

### Capture video from a webcam

In [ ]:
logging.basicConfig(level=logging.INFO)
# Variables 
PRAVEGA_GATEWAY = '10.1.83.104:30040'  # Pravega gateway
PRAVEGA_SCOPE = 'examples'
PRAVEGA_STREAM = 'video5'    
ADD_CHECKSUM = False  # Prepend the data with a checksum that can be used to detect errors    
USE_TRANSACTIONS = False  # If true, use Pravega transactions
MAX_CHUNK_SIZE = 1024*1024  # Maximum size of chunk (bytes)

# Capture video from camera with index 0
#cap = cv2.VideoCapture(0)
#logging.debug("frame width is {0}; frame height is {1}".format(cap.get(3), cap.get(4)))
#image_recorder = ImageRecorder(stream=camera)
with grpc.insecure_channel(PRAVEGA_GATEWAY) as pravega_channel:
    pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)
    pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=PRAVEGA_SCOPE))
    pravega_client.CreateStream(pravega.pb.CreateStreamRequest(scope=PRAVEGA_SCOPE, stream=PRAVEGA_STREAM))
    ssrc = np.random.randint(0, 2**31)
    data_iter = data_generator(image_recorder, ADD_CHECKSUM, ssrc)
    pravega_request_iter = pravega_request_generator(data_iter, PRAVEGA_SCOPE, PRAVEGA_STREAM, MAX_CHUNK_SIZE, USE_TRANSACTIONS)    
    write_response = pravega_client.WriteEvents(pravega_request_iter)
    logging.info("write_response=" + str(write_response))

# Realease the capture on exit
#cap.release()

INFO:root:0: {"timestamp": "2019-05-25T02:48:16.400Z", "frame_number": 0, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "payload_len": 786757, "final_chunk_index": 0}
INFO:root:0: {"timestamp": "2019-05-25T02:48:18.410Z", "frame_number": 1, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "payload_len": 786757, "final_chunk_index": 0}
INFO:root:0: {"timestamp": "2019-05-25T02:48:20.420Z", "frame_number": 2, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "payload_len": 786757, "final_chunk_index": 0}
INFO:root:0: {"timestamp": "2019-05-25T02:48:22.434Z", "frame_number": 3, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "payload_len": 786757, "final_chunk_index": 0}
INFO:root:0: {"timestamp": "2019-05-25T02:48:24.442Z", "frame_number": 4, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "payload_len": 786757, "final_chunk_index": 0}
INFO:root:0: {"timestamp": "2019-05-25T02:48:26.449Z", "frame_number": 5, "camera": 0, "ssrc": 1648338639, "data_len": 589987, "pa